In [1]:
import os
import sys
import json
import time
import math
import random
import datetime
from pathlib import Path
import wget

import argparse
import dill as pickle
from tqdm import tqdm
import urllib

import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch import Tensor
from torch.autograd import Variable
#from torchtext.data import Fields, Dataset, BucketIterator
#from torchtext.datasets import TranslationDataset
#import torchtext

import transformers
from transformers import BertModel
from transformers import BertConfig
from transformers import BertTokenizer

from PIL import Image
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
!nvidia-smi

Fri May 14 21:16:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   36C    P8    12W / 250W |    610MiB / 11264MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
print(torch.__version__)
print()
print(transformers.__version__)

1.8.1+cpu

4.5.1


In [5]:
#import sentencepiece as spm
from gluonnlp.data import SentencepieceTokenizer
from KoBERT.kobert.utils import get_tokenizer
tok_path = get_tokenizer()
sp  =  SentencepieceTokenizer(tok_path)
sp('한국어 모델을 공유합니다.')

using cached model


['▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.']

In [6]:
from transformers import BertConfig
from transformers import BertTokenizer
from transformers import BertForMaskedLM

config = BertConfig(
    config_size= 52000,
    max_position_embeddings=514,
    num_attention_heads= 12,
    num_hidden_layers=6,
    type_vocab_size=1
)

tokenizer = BertTokenizer.from_pretrained("bert-base-cased",max_len=512)

model = BertForMaskedLM(config=config)

In [7]:
model.num_parameters()

66987834

In [8]:

from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "./datasets/txtfiles/beatles.txt",
    block_size= 128
)
"""
from datasets import load_dataset
dataset = load_dataset('text',data_files={'train':'./datasets/WikiQACorpus/WikiQA-train.txt',
                      'test':'./datasets/WikiQACorpus/WikiQA-test.txt'})

from datasets import load_dataset
dataset = load_dataset('wikicorpus','raw_en')
"""

C:\Users\SCLAB\.conda\envs\cuda\lib\site-packages\transformers\data\datasets\language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  warnings.warn(


"\nfrom datasets import load_dataset\ndataset = load_dataset('text',data_files={'train':'./datasets/WikiQACorpus/WikiQA-train.txt',\n                      'test':'./datasets/WikiQACorpus/WikiQA-test.txt'})\n\nfrom datasets import load_dataset\ndataset = load_dataset('wikicorpus','raw_en')\n"

In [9]:
print(len(dataset))

1846


In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer= tokenizer, mlm=True, mlm_probability=0.15
)

In [11]:
print(data_collator)

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.15)


In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./BERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size= 4,
    save_steps=10000,
    save_total_limit=2,
    prediction_loss_only=True
)

#model = model.to('cuda:0')
trainer = Trainer(
    model = model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [13]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=462, training_loss=6.100174808915043, metrics={'train_runtime': 316.0885, 'train_samples_per_second': 1.462, 'total_flos': 13383365379192.0, 'epoch': 1.0})

# smallBERTa Pretraining

In [44]:
import pandas as pd

curdir = os.getcwd()
path1 = os.path.join(curdir,'datasets','twitter_samples','positive_tweets.json')
path2 = os.path.join(curdir,'datasets','twitter_samples','negative_tweets.json')

In [52]:
pos_df = pd.read_json(path1,lines=True)
neg_df = pd.read_json(path2,lines=True)

In [53]:
pos_df_text = pos_df['text']
neg_df_text = neg_df['text']

In [54]:
import regex as re

def basicPreprocess(text):
    try:
        processed_text = text.lower()
        processed_text = re.sub(r'\W +',' ',processed_text)
    except Exception as e:
        print("Exception:",e,",on text:",text)
        return None
    return processed_text

In [55]:
type(pos_df_text)

pandas.core.series.Series

In [56]:
pos_df_text = pos_df_text.apply(basicPreprocess).dropna()
neg_df_text = neg_df_text.apply(basicPreprocess).dropna()

In [58]:
pos_df_text = pos_df_text.replace('\n'," ")
neg_df_text = neg_df_text.replace('\n'," ")

In [69]:
txt_files_dir = "\\tmp\\text_split"
!mkdir {txt_files_dir}
i=0
for row in tqdm(pos_df_text.to_list()):
    file_name = os.path.join(txt_files_dir,str(i)+'.txt')
    try:
        f=open(file_name,'w')
        f.write(row)
        f.close()
    except Exception as e:
        print(row, e)
    i+=1

  5%|▍         | 241/5000 [00:00<00:04, 1152.48it/s]

💅🏽  :))) haven't seen you in years 'cp949' codec can't encode character '\U0001f485' in position 0: illegal multibyte sequence
spiritual ritual festival (népal)
beginning of line-up :)
it is left for the line-up (y)
see more at:.. http://t.co/qmnz62oeuc 'cp949' codec can't encode character '\xe9' in position 28: illegal multibyte sequence
@coffeebreak22 @laurenarren @drparkave @jwkanyuira1969 @dheracing @rhoachri @barbarabathurst @1cryingeye @willisteam : ☕☕ thank you rita! 'cp949' codec can't encode character '\u2615' in position 119: illegal multibyte sequence


 10%|▉         | 486/5000 [00:00<00:03, 1184.50it/s]

@literalwt lmaooo same its so hot 😤 like louis : he's so hot !!!!!!!! 'cp949' codec can't encode character '\U0001f624' in position 34: illegal multibyte sequence
@messiholic_ lol 😄😄really i can't believe a beautiful girl like you are single :p 'cp949' codec can't encode character '\U0001f604' in position 17: illegal multibyte sequence
@jackgilinsky hiii jack : ily ✨✨✨ 'cp949' codec can't encode character '\u2728' in position 30: illegal multibyte sequence
@fwmkian happy birthday baby 💟 i miss you so much and love you so much :)) 'cp949' codec can't encode character '\U0001f49f' in position 29: illegal multibyte sequence
tykkäsin @youtube-videosta http://t.co/p5dakaodmx 200sub special!  15e paysafecard giveaway! : (lue desc!!!) 'cp949' codec can't encode character '\xe4' in position 4: illegal multibyte sequence
i have ariana grande selena gomez louis tomlinson rita liam payne 
caradelevingne 🌷i trade solo dm :) 'cp949' codec can't encode character '\U0001f337' in position 83: illegal 

 15%|█▍        | 730/5000 [00:00<00:03, 1194.96it/s]

...

@briones198 always makes things better :)❤️ 'cp949' codec can't encode character '\u2764' in position 41: illegal multibyte sequence
@vivienneclore that is so cute :- 🐶 'cp949' codec can't encode character '\U0001f436' in position 34: illegal multibyte sequence
@stuarty2112 :- cheers pete will have a listen later ☺☺ 'cp949' codec can't encode character '\u263a' in position 53: illegal multibyte sequence
@mehra_sachi aha that's alright ❤️ we love you too :  a 'cp949' codec can't encode character '\u2764' in position 32: illegal multibyte sequence
smile its sunnah :)

#غردلي #عن http://t.co/levvielarv 'cp949' codec can't encode character '\u063a' in position 24: illegal multibyte sequence
hey @joshdevinedrums!:)

check this pictures out and what do you think?:)

notice and follow me please?💕

x http://t.co/iamfvdrmjb 'cp949' codec can't encode character '\U0001f495' in position 107: illegal multibyte sequence
party at our 3d printer : celebrating future technology's and it's possibilitie

 94%|█████████▍| 4700/5000 [00:03<00:00, 1204.69it/s]

happy #friday folks! if your order your #goodyear #tyres online @protyre you can get upto £40 cashback! (t&amp;cs apply #yourewelcome :) 'cp949' codec can't encode character '\xa3' in position 90: illegal multibyte sequence
its leo season :- ♌️ 'cp949' codec can't encode character '\u264c' in position 18: illegal multibyte sequence
@pussy_lilac no problem : it's a pleasure ❤️ 'cp949' codec can't encode character '\u2764' in position 42: illegal multibyte sequence
@notjagath are you a member of හෙල හවුල by any chance :d @chevindu 'cp949' codec can't encode character '\u0dc4' in position 31: illegal multibyte sequence
@demoorsophie hii can you follow me please i'd like ask you one thing in dm :)💞 'cp949' codec can't encode character '\U0001f49e' in position 78: illegal multibyte sequence
@iamahmadshahzad aameen..:)
long live pakistan..#bleedgreen..✌🏻️ 'cp949' codec can't encode character '\u270c' in position 62: illegal multibyte sequence
what could be more fun than a sexy girl two sexy 

100%|██████████| 5000/5000 [00:03<00:00, 1259.80it/s]

goodnight  i love luke with all my heart :- all my love 💜 'cp949' codec can't encode character '\U0001f49c' in position 56: illegal multibyte sequence
@_halie13_ thanks for being so great!: ❤️😘 'cp949' codec can't encode character '\u2764' in position 39: illegal multibyte sequence
@sivakaneswaran happy birthday  @jaymcguiness #jiva for ever : 🍹🍸🍻 'cp949' codec can't encode character '\U0001f379' in position 63: illegal multibyte sequence
jumma mubbarak : 
to all ❤❤ 'cp949' codec can't encode character '\u2764' in position 26: illegal multibyte sequence
@dontyk if it's no use to you i'll pick it up next time i'm down : yeah i'll cook in my nice big kitchen 😍😊 'cp949' codec can't encode character '\U0001f60d' in position 105: illegal multibyte sequence
    @louis_tomlinson    ⊹
thank you for always putting
a smile on my face
mind following me :)
have a nice day x
1710 'cp949' codec can't encode character '\u22b9' in position 24: illegal multibyte sequence
im going to bed i love hailey a

In [70]:
txt_files_dir = curdir+"\\tmp2\\text_split"
!mkdir {txt_files_dir}
i=0
for row in tqdm(neg_df_text.to_list()):
    file_name = os.path.join(txt_files_dir,str(i)+'.txt')
    try:
        f=open(file_name,'w')
        f.write(row)
        f.close()
    except Exception as e:
        print(row, e)
    i+=1

  5%|▌         | 251/5000 [00:00<00:03, 1247.79it/s]

no assignment but we have project : really 😩 'cp949' codec can't encode character '\U0001f629' in position 43: illegal multibyte sequence
my $$$$2 dollar : 😭😭😭😭😭 http://t.co/oi0pygusdi 'cp949' codec can't encode character '\U0001f62d' in position 18: illegal multibyte sequence
@littlemix french mixers miss you so much : 💜 'cp949' codec can't encode character '\U0001f49c' in position 44: illegal multibyte sequence
@concept_nathan nathann i never got a chance to take a pic with you : got a hug tho pic next time i see you?😊 'cp949' codec can't encode character '\U0001f60a' in position 108: illegal multibyte sequence
@deedee_50fly foreals : a wesen uwesiti mj 😂😩 'cp949' codec can't encode character '\U0001f602' in position 43: illegal multibyte sequence
raining on me today : 

gym it is then 💪🏻🐒 'cp949' codec can't encode character '\U0001f4aa' in position 41: illegal multibyte sequence


 10%|█         | 502/5000 [00:00<00:03, 1247.26it/s]

my last cross country in ij was amazing!!!!!!!💥✨ can't bear to leave :- https://t.co/vkvoxiaeqn 'cp949' codec can't encode character '\U0001f4a5' in position 46: illegal multibyte sequence
this weather has me scrolling through my contacts of people i've curved on some "hey you 😉 sigh :( 'cp949' codec can't encode character '\U0001f609' in position 89: illegal multibyte sequence
@nnoonlight i missed you : it was okay lol i did nothing 😂 'cp949' codec can't encode character '\U0001f602' in position 57: illegal multibyte sequence
@cmpunk @mattjackson13 please please come back to wwe : تكفى 'cp949' codec can't encode character '\u062a' in position 56: illegal multibyte sequence
i want jack's follow so bad but i don't want to be annoying by spamming him : sooo @jackgilinsky follow me please ☺💜 'cp949' codec can't encode character '\u263a' in position 114: illegal multibyte sequence
@littlemix have fun in germany : i can't be there 😭 'cp949' codec can't encode character '\U0001f62d' in posit

 15%|█▍        | 744/5000 [00:00<00:03, 1221.33it/s]

...

missed 11:11 : btw my wish was for @carterreynolds to follow me #loveyoutilltheendcarter ☺ 'cp949' codec can't encode character '\u263a' in position 89: illegal multibyte sequence
first two days of katies summer and she's been back and forth to the doctor with suspect meningitis and viral tonsillitis : poorly girl 😷 'cp949' codec can't encode character '\U0001f637' in position 136: illegal multibyte sequence
@jaymcguiness happy birthday again and please come back to singapore :- 😿💓💓💕😚😚😚😚😚❤️❤️❤️❤️❤️ http://t.co/5liiqjqpbt 'cp949' codec can't encode character '\U0001f63f' in position 72: illegal multibyte sequence
this boy was snapchattimg me and i was happy bc friends then he ruined it when he said "you wanna have fun😏😏 :(( 'cp949' codec can't encode character '\U0001f60f' in position 106: illegal multibyte sequence
@jjjaneell hay idk baby : 😭😭😭😭😭😭😭😭😭😭 'cp949' codec can't encode character '\U0001f62d' in position 26: illegal multibyte sequence
@waniiamira ehem 😏 haha ala yeke : its okay

 97%|█████████▋| 4832/5000 [00:04<00:00, 1173.83it/s]

@junior_jones it was good as always 😀flying back today though :( 'cp949' codec can't encode character '\U0001f600' in position 36: illegal multibyte sequence
@candysalvatore you're right i'm very tired : good night my ally i love you ❤️ 'cp949' codec can't encode character '\u2764' in position 76: illegal multibyte sequence
😂😂😂thank you san😀"@__zanele i wanna change my avi but usanele :(" 'cp949' codec can't encode character '\U0001f602' in position 0: illegal multibyte sequence
when fave unfollows :(😩 @itisfurny http://t.co/hovqloktig 'cp949' codec can't encode character '\U0001f629' in position 22: illegal multibyte sequence
happy birthday bes @mahhriel_114 enjoy your day beb i love you miss you sooooo muchie : see you soon sana 💞🎈🎂🎉 http://t.co/c5oobr6ov9 'cp949' codec can't encode character '\U0001f49e' in position 106: illegal multibyte sequence
@msmeghanmakeup hope you're having fun at vidcon! rlly bummed i couldnt go :( love you sunshine 💛💛💛💛 'cp949' codec can't encode character

100%|██████████| 5000/5000 [00:04<00:00, 1170.04it/s]

@courtneyr_green awwwww my baby : 💕 'cp949' codec can't encode character '\U0001f495' in position 34: illegal multibyte sequence
stop being sour : it doesn't suit you 😂 https://t.co/nj0ziq14is 'cp949' codec can't encode character '\U0001f602' in position 38: illegal multibyte sequence
@lottystorer @charliee_catlin 😂😩 what a mess :( 'cp949' codec can't encode character '\U0001f602' in position 30: illegal multibyte sequence
@dongvvoo1122 😀 r u sure u want that b r u h we barely survive w them in tank tops :(( 'cp949' codec can't encode character '\U0001f600' in position 14: illegal multibyte sequence


In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument("--corpus_file",type=str)
parser.add_arguemnt("--vocab_size",type=int,default=32000)
parser.add_argument("--lmit_alphabet",type=int,default=6000)

args = parser.parse_args()

tokenizer_custom = sp # tokenizer = BertWordPieceTokenizer()

#tokenizer.train()
#tokenizer.save()

